# Определение FraudScore депозита
Фиатной команде нужен AntiFraudService для получения fraudScore текущего депозита <br>для дальнейшего принятия решения об Capture или его отклонении. <br>Это поможет нам уменьшить Fraud/sell ratio.

**Некоторые показатели, указывающие на мошенников**
- Несоответствие локации
- Крупные суммы транзакций
- Частые транзакции
- Изменение типа транзакций
- Подозрительные места совершения платежей
- Необычные временные рамки
- Повторяющиеся транзакции
- Несколько неудачных попыток авторизации
- Использование заблокированных карт
<br>Важно помнить, что каждый индивидуальный показатель не обязательно указывает <br>на мошенничество - обычно это сочетание нескольких факторов вызывает тревогу.


**Описание данных тестового набора данных**
- "Kyc_citizenship" - страна выдачи документа в kyc в формате ISO2;
- "Deposit_amount_cat" - категория сумма депозита в USD. <br>(≥0 до 100 - 1я категория. ≥100 до 500 - 2я, ≥500 до 2000 - 3я, ≥2000 - 4я);
- "Deposit_user_time_diff_cat" - категория разницы в днях между совершенным депозитом <br>и созданием пользователя (≥0 до 3 - 1я категория, ≥3 до 10 - 2я, ≥10 до 30 - 3я, ≥30 - 4я);
- "Deposit_kyc_time_diff_cat" - категория разницы в днях между совершенным депозитом и <br>прохождением пользователем kyc (<0 - 0я категория, ≥0 до 3 - 1я, ≥3 до 10 - 2я, ≥10 до 30 - 3я, ≥30 - 4я);
- "Deposit_card_country" - страна выпуска карты в формате ISO2;
- "Deposit_ip_country" - ip адрес с которого совершен депозит в формате ISO2;
- "Deposit_language" - выбранный язык на устройстве (en, ru … );
- "deposit_fraud" - целевой параметр. <br>Принимает значение true если deposit_fraud=true или deposit_disputed=true.

**Ход исследования**
- Знакомство с данными
- Предобработка данных (при необходимости)
- Тренировка трёх моделей
- Выбор лучшей модели
- Проверка модели на тестовой выборке
- Определение важности признаков
- Заключение

## Импорт библиотек

In [1]:
import pandas as pd

## EDA

Тренировочный набор данных

In [3]:
df_exm = pd.read_excel('data/Data set.xlsx')
df_exm.info()
df_exm.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   user_id                     4115 non-null   object
 1   kyc_citizenship             4051 non-null   object
 2   deposit_amount_cat          4115 non-null   int64 
 3   deposit_user_time_diff_cat  4115 non-null   int64 
 4   deposit_kyc_time_diff_cat   4115 non-null   int64 
 5   deposit_card_country        4037 non-null   object
 6   deposit_ip_country          4049 non-null   object
 7   deposit_language            4115 non-null   object
 8   deposit_fraud               4115 non-null   bool  
dtypes: bool(1), int64(3), object(5)
memory usage: 261.3+ KB


,user_id,kyc_citizenship,deposit_amount_cat,deposit_user_time_diff_cat,deposit_kyc_time_diff_cat,deposit_card_country,deposit_ip_country,deposit_language,deposit_fraud
3021,c2a1ce49-5b27-4b8d-b9c5-67c760cf4506,FR,2,4,0,FR,FR,fr,False
3943,f1b9606e-f646-491b-99a2-7e24314b5d1c,GR,2,2,2,GR,GR,el,False
4032,f7311a1e-e110-4df8-9fe8-6ee11ada6316,TN,1,4,1,FR,FR,fr,False
2973,bf214f2e-4b19-4efa-8a89-306fad0de757,GB,2,2,2,GB,GB,en,False
2073,7ad38166-50a7-4130-8ffb-2e8b5bd39871,GB,3,1,1,GB,GB,en,False


In [4]:
df_exm['deposit_fraud'].value_counts()

deposit_fraud
False    3918
True      197
Name: count, dtype: int64

Полный набор данных

In [2]:
df_full = pd.read_csv('data/DataSet_full.csv')
df_full.info()
df_full.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27803 entries, 0 to 27802
Data columns (total 19 columns):
 #   Column                                                                                           Non-Null Count  Dtype  
---  ------                                                                                           --------------  -----  
 0   001b8147-5eda-4b05-9f4a-bf43a56a617e                                                             27803 non-null  object 
 1   2023-08-10 12:59:50.679000                                                                       27803 non-null  object 
 2   ES                                                                                               27304 non-null  object 
 3   2023-08-10 13:10:18.565292                                                                       27803 non-null  object 
 4   ES.1                                                                                             27341 non-null  object 
 5   ES.2

,001b8147-5eda-4b05-9f4a-bf43a56a617e,2023-08-10 12:59:50.679000,ES,2023-08-10 13:10:18.565292,ES.1,ES.2,Unnamed: 6,2023-08-15 14:44:53,FAILED,The card was declined.,1150,EUR,MasterCard,ES.3,"""ipAddress"":""46.222.39.193"",""ipCountry"":""ES""","Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko)","es-ES,es;q=0.9",false,false.1
6519,37065f49-f5d2-42eb-b95f-4bdeaf5248e9,2021-11-26 19:32:14.658000,ZA,2022-01-17 14:30:16.512056,ZA,ZA,NaN,2023-05-13 13:32:44,FAILED,Card failed 3d secure,NaN,NaN,MasterCard,ZA,"""ipAddress"":""105.245.160.44"",""ipCountry"":""ZA""",Mozilla/5.0 (Linux; Android 11; SM-A107F Build...,"en,en-ZA;q=0.9,en-US;q=0.8",NaN,NaN
9635,50a52d8a-5ce9-419b-b519-bf89edb430fa,2023-05-26 15:53:21.424000,GB,2023-05-26 16:26:59.839292,RU,RU,NaN,2023-06-11 17:24:34,FAILED,Card failed 3d secure,NaN,NaN,MasterCard,GB,"""ipAddress"":""94.41.167.9"",""ipCountry"":""RU""",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"ru,en;q=0.9",NaN,False
2610,151c6e34-da7a-462d-94ff-8c2864948405,2022-03-23 16:33:25.687000,DE,2023-08-14 18:14:08.893742,CM,DE,NaN,2023-08-14 18:26:21,FAILED,Card failed 3d secure,NaN,NaN,Visa,BG,"""ipAddress"":""2a02:3031:207:68da:f1ac:88b:2670:...",Mozilla/5.0 (Linux; Android 13; SM-G990B2 Buil...,"en,de-DE;q=0.9,de;q=0.8,en-GB;q=0.7,fr-FR;q=0....",False,False
11407,6187e492-be90-474c-828b-bb7482ab12f6,2023-06-08 02:05:53.898000,AU,2023-06-08 03:14:01.086864,AU,AU,NaN,2023-10-06 01:19:00,FAILED,Card failed 3d secure,NaN,NaN,MasterCard,AU,"""ipAddress"":""49.185.142.220"",""ipCountry"":""AU""",Mozilla/5.0 (Linux; Android 11; P652 Build/RP1...,"en,en-AU;q=0.9,en-US;q=0.8",False,False
9830,5349d546-b201-468f-bc45-7a53d5f5b453,2022-02-12 08:24:10.083000,ZA,2022-02-25 09:56:21.015997,ZA,ZA,NaN,2023-11-28 05:21:06,FAILED,One user purchases after the daily max amount ...,15.0,EUR,MasterCard,ZA,"""ipAddress"":""165.0.139.197"",""ipCountry"":""ZA""",Mozilla/5.0 (Linux; Android 8.1.0; DUB-LX1 Bui...,"en,en-ZA;q=0.9,en-US;q=0.8",False,False
